# <center>Design

## [146. LRU缓存机制 (medium)](https://leetcode-cn.com/problems/lru-cache/)

运用你所掌握的数据结构，设计和实现一个```LRU (最近最少使用) 缓存机制```。它应该支持以下操作： 获取数据 ```get``` 和 写入数据 ```put``` 。

- 获取数据 ```get(key)``` - 如果关键字 (key) 存在于缓存中，则获取关键字的值（总是正数），否则返回 -1。
- 写入数据 ```put(key, value)``` - 如果关键字已经存在，则变更其数据值；如果关键字不存在，则插入该组「关键字/值」。当缓存容量达到上限时，它应该在写入新数据之前删除最久未使用的数据值，从而为新的数据值留出空间。

在 **O(1)** 时间复杂度内完成这两种操作？

<font color='dd0000'>

要记录键值对，首先需要一个哈希表。

要能记录数据的访问顺序，想到用一个链表/队列来保存数据，头部保存较新的数据，尾部保存较旧的数据。

但对刚刚访问过的数据，要在O(1)时间内调整它在链表中的位置。如果是单独的链表是无法实现的，所以应**用哈希表的key映射到链表的节点**，这样就能在O(1)时间定位到刚刚访问到的数据。又要调整这个数据节点在链表中的位置，所以要使用**双向链表**。

为了避免边界判空，建立```dummy head```和```dummy tail```

</font>

In [ ]:
class LRUCache {
    Map<Integer, Node> map;
    Node head;  // 靠近head的是更新的节点
    Node tail;  // 靠近tail的是更旧的节点
    int size;
    int capacity;

    public LRUCache(int capacity) {
        map = new HashMap<>();
        head = new Node(-1, -1);
        tail = new Node(-1, -1);
        head.before = tail;
        head.after = tail;
        tail.before = head;
        tail.after = head;
        size = 0;
        this.capacity = capacity;
    }
    
    public int get(int key) {
        if(map.containsKey(key)){
            Node node = map.get(key);
            adjust(node);  // 将最近访问的节点调整到头部
            return node.value;
        }
        return -1;
    }
    
    public void put(int key, int value) {
        if(map.containsKey(key)){   // 如果key已存在，旧覆盖value值，且把节点调整到头部
            Node node = map.get(key);
            node.value = value;
            adjust(node);
            return;
        } else if(size == capacity){ // key不存在，但容量已满，删去最旧的节点，即tail的前一个节点
            Node del = tail.before;
            delete(del);      // 从双向链表中删去
            map.remove(del.key);  // 从哈希表中删去
        }
        Node newNode = new Node(key, value); // new一个新节点
        insert(newNode);    // 插入双向链表
        map.put(key, newNode); // 插入哈希表
    }

    private void adjust(Node node){  // 调整节点到头部
        node.before.after = node.after;
        node.after.before = node.before;
        node.after = head.after;
        head.after.before = node;
        node.before = head;
        head.after = node;
    }

    private void delete(Node node){ // 删除最旧的节点
        node.before.after = tail;
        tail.before = node.before;
        node.after = null;
        node.before = null;
        size--;
    }

    private void insert(Node node){ // 插入新节点到头部
        node.after = head.after;
        node.after.before = node;
        node.before = head;
        head.after = node;
        size++;
    }
}

class Node{    // 双向链表的节点，数据域要包括key，因为涉及到从map中删去最旧的节点，如果没有key无法调用map.remove(key)
    Node before;
    Node after;
    int value;
    int key;

    public Node(int key, int value){
        this.key = key;
        this.value = value;
    }
}

## [460. LFU缓存 (hard)](https://leetcode-cn.com/problems/lfu-cache/)

请你为 ```最不经常使用（LFU）```缓存算法设计并实现数据结构。它应该支持以下操作：```get``` 和 ```put```。

- ```get(key)``` - 如果键存在于缓存中，则获取键的值（总是正数），否则返回 -1。
- ```put(key, value)``` - 如果键已存在，则变更其值；如果键不存在，请插入键值对。当缓存达到其容量时，则应该在插入新项之前，使最不经常使用的项无效。在此问题中，当存在平局（即两个或更多个键具有相同使用频率）时，应该去除最久未使用的键。

「项的使用次数」就是自插入该项以来对其调用 get 和 put 函数的次数之和。使用次数会在对应项被移除后置为 0 。

在 **O(1)** 时间复杂度内执行两项操作

<font color='dd0000'>

和上题相比，不仅需要对访问顺序排序，而且应该对数据的访问频率排序，且频率优先级高于访问顺序

要在O(1)时间内完的操作有：定位到数据点、改变频率域、调整顺序。并且顺序须先按频率排，再按访问顺序排

按访问顺序排，使用双向链表可以完成操作

所以，在原有实现缓存的 Map 基础上，增加一个**键为频率，值为双向链表**的 Map 。这样就能同时完成对频率和访问顺序都排序

在每个频率对应的双向链表中，靠近头节点的是更新的节点

同时，需要一个始终记录最低频率的变量，用来移除

</font>

In [2]:
// O(1)时间复杂度

class Node{  // 双向链表的节点，用来作为数据点，保存 key(从cache中移除时用)、value、freq(访问频率)
    
    int key;
    int value;
    int freq;
    Node before;
    Node after;

    public Node(int key, int value, int freq){
        this.key = key;
        this.value = value;
        this.freq = freq;
    }
}

class DoubleLinkedList{  // 双向链表，作为 freqMap 的值，一个频率对应一个链表，方便同时对频率和访问顺序进行排序
    
    Node head;  // 虚拟的头尾节点，避免判空
    Node tail;

    public DoubleLinkedList(){
        head = new Node(-1, -1, -1);
        tail = new Node(-1, -1, -1);
        head.after = tail;
        head.before = tail;
        tail.after = head;
        tail.before = head;
    }

    public boolean isEmpty(){   // 判空方法
        return head.after == tail;
    }

    public void delete(Node delNode){  // 从双向链表中删除节点
        delNode.after.before = delNode.before;
        delNode.before.after = delNode.after;
        delNode.after = null;
        delNode.before = null;
    }

    public void addFirst(Node newNode){  // 增加节点到头部，头部表示在同一频率下更新的节点
        newNode.after = head.after;
        newNode.before = head;
        head.after.before = newNode;
        head.after = newNode;
    }
}

class LFUCache {
    
    Map<Integer, Node> cache;  // 通过key访问到链表中的数据
    Map<Integer, DoubleLinkedList> freqMap; // 实际保存数据的结构，一个频率对应一个双向链表
    int minFreq;  // 记录最小频率
    int size;
    int capacity;

    public LFUCache(int capacity) {
        cache = new HashMap<>();
        freqMap = new HashMap<>();
        DoubleLinkedList ini = new DoubleLinkedList();
        freqMap.put(1, ini);  // 初始化一个频率 = 1 对应的链表，因为只要有节点加入，频率就为1，直接创建出来
        minFreq = 1;  // 初始最小频率 = 1
        size = 0;
        this.capacity = capacity;
    }
    
    public int get(int key) {
        if(cache.containsKey(key)){
            Node cur = cache.get(key); // 获得数据节点
            adjust(cur);  // 访问频率+1， 调整这个节点在链表中的位置
            return cur.value;
        }
        return -1;
    }
    
    public void put(int key, int value) {
        if(capacity == 0){
            return;
        }
        if(cache.containsKey(key)){
            Node cur = cache.get(key); // 节点已存在，直接修改
            cur.value = value;
            adjust(cur);   // 同时访问频率+1，调整在链表中的位置
            return;
        }
        if(size == capacity){
            DoubleLinkedList curList = freqMap.get(minFreq);  // 用最小频率定位到对应的链表
            Node delNode = curList.tail.before;  // 这个链表的最后一个节点就是频率最低且最久未使用的节点
            int k = delNode.key;
            cache.remove(k);  // 从cache中移除
            curList.delete(delNode); // 同时从链表中移除
            size--;
        }
        Node newNode = new Node(key, value, 1); // 新节点
        cache.put(key, newNode);
        DoubleLinkedList curList = freqMap.get(1);  // 新节点一开始频率为1，插入1对应的链表
        curList.addFirst(newNode);
        minFreq = 1;   // 有新节点插入时，最小频率一定为1
        size++;
    }

    private void adjust(Node cur){  // 当有访问增加频率时，对节点进行操作
        int preFreq = cur.freq;
        DoubleLinkedList oldList = freqMap.get(preFreq); // 从旧频率链表中删除该节点
        oldList.delete(cur);
        cur.freq++;  // 频率+1
        if(!freqMap.containsKey(cur.freq)){   // 如果+1后的频率对应的链表还不存在，就创建出来
            DoubleLinkedList newList = new DoubleLinkedList();
            freqMap.put(cur.freq, newList);
        }
        DoubleLinkedList curList = freqMap.get(cur.freq);  // 把节点加在新频率链表的头部
        curList.addFirst(cur);
        if(oldList.isEmpty() && minFreq == preFreq){  // 如果旧频率是最小频率且是唯一的节点，那么最小频率就要+1
            minFreq++;
        }
    }
}

<font color='dd0000'>使用 TreeMap ，可以有 $log(capacity)$ 复杂度的算法。将节点插在 TreeMap 中，按照先频率后访问时间的顺序排序</font>

In [3]:
// O(log(n))时间复杂度

class Node implements Comparable<Node>{  // 数据节点，重写compareTo方法，用于在TreeMap中排序
    int key;
    int value;
    int frequency;  // 表示访问频率
    int timestamp;  // 时间戳，用来表示访问顺序

    public Node(int key, int value, int timestamp){
        this.value = value;
        this.key = key;
        this.timestamp = timestamp;
        frequency = 0;
    }

    public int compareTo(Node node){  // 先频率后时间排序
        if(this.frequency == node.frequency){
            return this.timestamp - node.timestamp;
        }
        return this.frequency - node.frequency;
    }
}

class LFUCache {
    HashMap<Integer, Node> map; // key到数据点的映射
    TreeSet<Node> tree; // 实际存储结构
    int time;  // 时间戳
    int capacity;

    public LFUCache(int capacity) {
        this.capacity = capacity;
        map = new HashMap<>();
        tree = new TreeSet<>();
        time = 0;
    }
    
    public int get(int key) {
        if(map.containsKey(key)){
            Node cur = map.get(key);  // 取出当前节点，从TreeMap中移除，增加它的频率和时间，再插回TreeMap
            tree.remove(cur);
            cur.frequency++;
            cur.timestamp = ++time;
            tree.add(cur);
            return cur.value;
        }
        return -1;
    }
    
    public void put(int key, int value) {
        if (capacity == 0)
            return;

        if (map.containsKey(key)) { // 取出当前节点，从TreeMap中移除，改变value值，增加它的频率和时间，再插回TreeMap
            Node temp = map.get(key);
            tree.remove(temp);
            temp.value = value;
            temp.frequency++;
            temp.timestamp = ++time;
            tree.add(temp);
            return;
        }

        if (map.size() == capacity) {
            Node del = tree.pollFirst();  // 移除频率最小(最旧)的点
            int k = del.key;
            map.remove(k);
        }
        Node newNode = new Node(key, value, ++time); // 插入新点
        map.put(key, newNode);
        tree.add(newNode);
    }
}